Essential libraries required for training and data loading.

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import csv
import pandas as pd

Loading phi-1.5 model from hugging face.

In [2]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [3]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Adding tokens for conversation style training.

In [4]:
tokenizer.bos_token = "<s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<|user|>","<end>","<|end|>","<|assistant|>"])

4

Adding chat template for conversation style training.

In [5]:
tokenizer.chat_template = """{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') %}{{'<|user|>' + '
' + message['content'] + '<|end|>' + '
' + '<|assistant|>' + '
'}}{% elif (message['role'] == 'assistant') %}{{message['content'] + '<|endoftext|>'}}{% endif %}{% endfor %}"""

Loading training data.

In [6]:
!wget https://huggingface.co/datasets/Kaludi/Customer-Support-Responses/resolve/main/Customer-Support.csv

--2024-06-24 15:54:37--  https://huggingface.co/datasets/Kaludi/Customer-Support-Responses/resolve/main/Customer-Support.csv
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12434 (12K) [text/plain]
Saving to: ‘Customer-Support.csv’

Customer-Support.cs 100%[===================>]  12.14K  --.-KB/s    in 0s      

2024-06-24 15:54:37 (300 MB/s) - ‘Customer-Support.csv’ saved [12434/12434]



In [7]:
df = pd.read_csv("Customer-Support.csv", sep=",")

In [8]:
df.head()

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...


Checking chat template.

In [9]:
messages =[
    {"role":"user",
     "content": "what is happening?"},
    {"role":"assistant",
     "content": "its life."}
]
print(tokenizer.apply_chat_template(messages,tokenize=False))
tokenizer(tokenizer.apply_chat_template(messages,tokenize=False),return_tensors="pt")


<s><|user|>
what is happening?<|end|>
<|assistant|>
its life.<|endoftext|>


{'input_ids': tensor([[   27,    82,    29, 50295,   198, 10919,   318,  5836,    30, 50297,
           198, 50298,   198,   896,  1204,    13, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Dataset for training pytorch model.

In [10]:
class Batch_D(nn.Module):
  def __init__(self):
    super().__init__()

class MyDataset(Dataset):
  def __init__(self,file_path,tokenizer):
    super().__init__()
    self.data = pd.read_csv("Customer-Support.csv", sep=",")
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    query = self.data["query"][idx]
    response = self.data["response"][idx]

    messages =[
      {"role":"user",
      "content": query},
      {"role":"assistant",
      "content": response}
    ]
    text = self.tokenizer.apply_chat_template(messages,tokenize=False)

    return text



class MyCollate():
  def __init__(self,tokenizer):
    self.tokenizer = tokenizer

  def __call__(self,batch):
    tokenized = self.tokenizer(batch,return_tensors="pt",padding=True)
    batch_data = Batch_D()
    batch_data.input_ids = tokenized.input_ids
    batch_data.attention_mask = tokenized.attention_mask
    batch_data.labels = tokenized.input_ids
    return batch_data





In [11]:
mydataset = MyDataset("Customer-Support.csv",tokenizer)
mycollate = MyCollate(tokenizer)


Training loops.

In [12]:
def train_epoch(model,train_dataloader,optimizer,config):
  i =0
  total = len(train_dataloader)
  for bd in train_dataloader:
    output = model(input_ids=bd.input_ids.to(config.device),attention_mask=bd.attention_mask.to(config.device),labels=bd.labels.to(config.device))
    loss = output.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("loss",loss,f"{i}/{total}")
    i+= 1

def train(model,train_dataloader,config):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.999), eps=1e-08)
  for epoch in range(config.max_epoch):
    train_epoch(model,train_dataloader,optimizer,config)


Setting config like batch_size and max epoch.

In [16]:
config = Batch_D()
config.max_epoch = 4
config.batch_size = 4
config.device = "cpu"


Creating dataloader.

In [17]:
mydataloader = DataLoader(mydataset, batch_size=config.batch_size, shuffle=True, collate_fn=mycollate)

start training.

In [18]:
model.to(config.device)
train(model,mydataloader,config)

loss tensor(4.1699, grad_fn=<NllLossBackward0>) 0/19
loss tensor(3.2304, grad_fn=<NllLossBackward0>) 1/19
loss tensor(3.0600, grad_fn=<NllLossBackward0>) 2/19
loss tensor(2.5182, grad_fn=<NllLossBackward0>) 3/19
loss tensor(2.1432, grad_fn=<NllLossBackward0>) 4/19
loss tensor(1.8802, grad_fn=<NllLossBackward0>) 5/19
loss tensor(2.1015, grad_fn=<NllLossBackward0>) 6/19
loss tensor(1.9302, grad_fn=<NllLossBackward0>) 7/19
loss tensor(1.8086, grad_fn=<NllLossBackward0>) 8/19
loss tensor(1.8624, grad_fn=<NllLossBackward0>) 9/19
loss tensor(1.5042, grad_fn=<NllLossBackward0>) 10/19
loss tensor(1.5024, grad_fn=<NllLossBackward0>) 11/19
loss tensor(1.4162, grad_fn=<NllLossBackward0>) 12/19
loss tensor(1.5803, grad_fn=<NllLossBackward0>) 13/19
loss tensor(1.4442, grad_fn=<NllLossBackward0>) 14/19
loss tensor(1.5914, grad_fn=<NllLossBackward0>) 15/19
loss tensor(1.4678, grad_fn=<NllLossBackward0>) 16/19
loss tensor(1.2734, grad_fn=<NllLossBackward0>) 17/19
loss tensor(1.2284, grad_fn=<NllLossBa

loading generation config for output generation.

In [19]:
torch.save(model.state_dict(), "model.pt")

In [20]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=100, do_sample=True,eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id
)

demo function for getting response.

In [23]:
def demo(query):
  messages =[
    {"role":"user",
     "content": query}
    ]
  text = tokenizer.apply_chat_template(messages,tokenize=False)
  inputs = tokenizer(text,return_tensors="pt",return_attention_mask=False).to(config.device)
  outputs = model.generate(**inputs, generation_config=generation_config)
  answer = tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  return answer.split("<|assistant|>\n")[1]



You can change query.

In [29]:
query = "I want to change my shipping address."
demo(query)

"No problem. Can you please provide your order number and the new shipping address you'd like to use?"

In [25]:
df.head()

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...
